In [1]:
%load_ext sql

In [2]:
%sql sqlite:///myfile.db

In [4]:
%%sql
-- Select the count of each level of priority
SELECT priority, count(*) 
  FROM evanston311
 GROUP BY priority;

 * sqlite:///myfile.db
Done.


priority,count(*)
HIGH,88
LOW,517
MEDIUM,5745
NONE,30081


In [5]:
%%sql
-- Find values of zip that appear in at least 100 rows
-- Also get the count of each value
SELECT zip, count(*) 
  FROM evanston311
 GROUP BY zip
HAVING count(*) >=100; 


 * sqlite:///myfile.db
Done.


zip,count(*)
None,5528
60201.0,19054
60202.0,11165
60208.0,255


In [6]:
%%sql
-- Find values of source that appear in at least 100 rows
-- Also get the count of each value
SELECT source, count(*) 
  FROM evanston311
 GROUP BY source
HAVING count(*) >=100; 

 * sqlite:///myfile.db
Done.


source,count(*)
Android,444
Iframe,3670
gov.publicstuff.com,30985
iOS,1199


In [7]:
%%sql
-- Find the 5 most common values of street and the count of each
SELECT street, count(*) 
  FROM evanston311
 GROUP BY street
 ORDER BY count(*) DESC
 LIMIT 5;



 * sqlite:///myfile.db
Done.


street,count(*)
None,1699
Chicago Avenue,1440
Sherman Avenue,1276
Central Street,1211
Davis Street,1154


In [9]:
%%sql
SELECT distinct street,
       -- Trim off unwanted characters from street
       trim(street, '0123456789 #/.') AS cleaned_street
  FROM evanston311
 ORDER BY street limit 4;



 * sqlite:///myfile.db
Done.


street,cleaned_street
None,None
1/2 Chicago Ave,Chicago Ave
1047B Chicago Ave,B Chicago Ave
13th Street,th Street


In [10]:
%%sql
-- Count rows
SELECT count(*)
  FROM evanston311
 -- Where description includes trash or garbage
 WHERE description ILIKE '%trash%' 
    OR description ILIKE '%garbage%' limit 5;

 * sqlite:///myfile.db
(sqlite3.OperationalError) near "ILIKE": syntax error
[SQL: -- Count rows
SELECT count(*)
  FROM evanston311
 -- Where description includes trash or garbage
 WHERE description ILIKE '%trash%' 
    OR description ILIKE '%garbage%' limit 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [11]:
%%sql
-- Count rows
SELECT count(*) 
FROM evanston311 
-- Where description includes trash or garbage
WHERE description LIKE '%trash%' 
  OR description LIKE '%garbage%' 
LIMIT 5;

 * sqlite:///myfile.db
Done.


count(*)
2551


In [13]:
%%sql
-- Select categories containing Trash or Garbage
SELECT category
  FROM evanston311
 -- Use LIKE
 WHERE  category LIKE '%Trash%'
    or category LIKE '%Garbage%' limit 5;

 * sqlite:///myfile.db
Done.


category
THIS REQUEST IS INACTIVE...Trash Cart - Compost Bin
Trash - Tire Pickup
Trash - Special Pickup - Resident Use
"Trash, Recycling, Yard Waste Cart- Repair/Replacement"
"Trash, Recycling, Yard Waste Cart- Repair/Replacement"


In [14]:
%%sql
-- Count rows
SELECT count(*)
  FROM evanston311 
 -- description contains trash or garbage (any case)
 WHERE (description LIKE '%Trash%'
    or description LIKE '%Garbage%') 
 -- category does not contain Trash or Garbage
   AND category NOT LIKE '%Trash%'
   AND category NOT LIKE '%Garbage%';

 * sqlite:///myfile.db
Done.


count(*)
570


In [17]:
%%sql
-- Count rows with each category
SELECT category, count(*)
  FROM evanston311 
 WHERE (description LIKE '%trash%'
    OR description LIKE '%garbage%') 
   AND category NOT LIKE '%Trash%'
   AND category NOT LIKE '%Garbage%'
 -- What are you counting?
 GROUP BY category
 --- order by most frequent values
 ORDER BY count(*) DESC
 LIMIT 10;

 * sqlite:///myfile.db
Done.


category,count(*)
Ask A Question / Send A Message,273
Rodents- Rats,77
Recycling - Missed Pickup,28
Dead Animal on Public Property,16
Graffiti,15
Yard Waste - Missed Pickup,14
Public Transit Agency Issue,13
Food Establishment - Unsanitary Conditions,13
Exterior Conditions,10
Street Sweeping,9


In [18]:
%%sql
-- Concatenate house_num, a space, and street
-- and trim spaces from the start of the result
SELECT ltrim(concat(house_num, ' ', street)) AS address
  FROM evanston311;

 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: concat
[SQL: -- Concatenate house_num, a space, and street
-- and trim spaces from the start of the result
SELECT ltrim(concat(house_num, ' ', street)) AS address
  FROM evanston311;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
%%sql
-- Concatenate house_num, a space, and street
-- and trim spaces from the start of the result
SELECT ltrim(house_num || ' ' || street) AS address
FROM evanston311 limit 5;

 * sqlite:///myfile.db
Done.


address
606-612 Sheridan Road
930 Washington St
1183-1223 Lincoln St
1–111 Callan Ave
1524 Crain St


In [21]:
%%sql
-- Select the first word of the street value
SELECT split_part(street, ' ', 1) AS street_name, 
       count(*)
  FROM evanston311
 GROUP BY street_name
 ORDER BY count DESC
 LIMIT 5;

 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: split_part
[SQL: -- Select the first word of the street value
SELECT split_part(street, ' ', 1) AS street_name, 
       count(*)
  FROM evanston311
 GROUP BY street_name
 ORDER BY count DESC
 LIMIT 25;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [24]:
%%sql
-- Select the first word of the street value
SELECT substr(street,1,instr(street,' ')-1) AS street_name,
       count(*)
FROM evanston311
GROUP BY street_name
ORDER BY count(*) DESC
LIMIT 5;

 * sqlite:///myfile.db
Done.


street_name,count(*)
None,1699
Chicago,1569
Central,1529
Sherman,1479
Davis,1248


In [26]:
%%sql
-- Select the first 50 chars when length is greater than 50
SELECT CASE WHEN length(description) > 50
            THEN left(description, 50) || '...'
       -- otherwise just select description
       ELSE description
       END
  FROM evanston311
 -- limit to descriptions that start with the word I
 WHERE description LIKE 'I %'
 ORDER BY description;

 * sqlite:///myfile.db
(sqlite3.OperationalError) no such function: left
[SQL: -- Select the first 50 chars when length is greater than 50
SELECT CASE WHEN length(description) > 50
            THEN left(description, 50) || '...'
       -- otherwise just select description
       ELSE description
       END
  FROM evanston311
 -- limit to descriptions that start with the word I
 WHERE description LIKE 'I %'
 ORDER BY description;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [28]:
%%sql
SELECT CASE WHEN length(description) > 50
            THEN substr(description, 1, 50) || '...'
       -- otherwise just select description
       ELSE description
       END
  FROM evanston311
 -- limit to descriptions that start with the word I
 WHERE description LIKE 'I %'
 ORDER BY description limit 6;

 * sqlite:///myfile.db
Done.


"CASE WHEN length(description) > 50 THEN substr(description, 1, 50) || '...' -- otherwise just select description ELSE description END"
I work for Schermerhorn & Co. and manage this con...
"I Live in a townhouse with garbage cans in back, i..."
"I Put In For Reserve Disabled Parking, A Week Ago ..."
I SDO GOWANS #1258 RECEIVED A TELEPHONE CALL ON 3/...
I accidentally mistyped my license plate number - ...
I accidentally sent the wrong cover letter on my a...


In [29]:
%%sql
-- Fill in the command below with the name of the temp table
DROP TABLE IF EXISTS recode;

-- Create and name the temporary table
CREATE TEMP TABLE recode AS
-- Write the select query to generate the table 
-- with distinct values of category and standardized values
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
    -- What table are you selecting the above values from?
    FROM evanston311;
       
-- Look at a few values before the next step
SELECT DISTINCT standardized 
  FROM recode
 WHERE standardized LIKE 'Trash%Cart'
    OR standardized LIKE 'Snow%Removal%';

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) no such function: split_part
[SQL: -- Create and name the temporary table
CREATE TEMP TABLE recode AS
-- Write the select query to generate the table 
-- with distinct values of category and standardized values
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
    -- What table are you selecting the above values from?
    FROM evanston311;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
%%sql
-- Create and name the temporary table
CREATE TEMP TABLE recode AS
-- Write the select query to generate the table 
-- with distinct values of category and standardized values
SELECT DISTINCT category, 
       substr(category, 1, instr(category, '-') - 1) AS standardized
FROM evanston311;

In [40]:
%%sql
select * from recode limit 5;

 * sqlite:///myfile.db
Done.


category,standardized
Fire Prevention - Inspection of a Commercial Property,Fire Prevention
Water Service - Question or Concern,Water Service
Trees-Fallen limb or tree,Trees
Ask A Question / Send A Message,
Dead Animal on Public Property,


In [41]:
%%sql
-- Code from previous step
DROP TABLE IF EXISTS recode;

CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
    FROM evanston311;
  
-- Update to group trash cart values
UPDATE recode 
   SET standardized='Trash Cart' 
 WHERE standardized LIKE 'Trash%Cart';

-- Update to group snow removal values
UPDATE recode
   SET standardized='Snow Removal' 
 WHERE standardized LIKE 'Snow%Removal%';
 
-- Examine effect of updates
SELECT DISTINCT standardized 
  FROM recode
 WHERE standardized LIKE 'Trash%Cart'
    OR standardized LIKE 'Snow%Removal%';

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) no such function: split_part
[SQL: CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
    FROM evanston311;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [42]:
%%sql
-- Code from previous step
DROP TABLE IF EXISTS recode;
CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;
UPDATE recode SET standardized='Trash Cart' 
 WHERE standardized LIKE 'Trash%Cart';
UPDATE recode SET standardized='Snow Removal' 
 WHERE standardized LIKE 'Snow%Removal%';
UPDATE recode SET standardized='UNUSED' 
 WHERE standardized IN ('THIS REQUEST IS INACTIVE...Trash Cart', 
               '(DO NOT USE) Water Bill',
               'DO NOT USE Trash', 'NO LONGER IN USE');

-- Select the recoded categories and the count of each
SELECT standardized, count(*) 
-- From the original table and table with recoded values
  FROM evanston311 
       LEFT JOIN recode 
       -- What column do they have in common?
       ON evanston311.category=recode.category 
 -- What do you need to group by to count?
 GROUP BY standardized
 -- Display the most common val values first
 ORDER BY count DESC;

 * sqlite:///myfile.db
Done.
(sqlite3.OperationalError) no such function: split_part
[SQL: CREATE TEMP TABLE recode AS
  SELECT DISTINCT category, 
         rtrim(split_part(category, '-', 1)) AS standardized
  FROM evanston311;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [46]:
%%sql
-- To clear table if it already exists
DROP TABLE IF EXISTS indicators;

-- Create the indicators temp table
CREATE TEMP TABLE indicators AS
  -- Select id
  SELECT id, 
         -- Create the email indicator (find @)
         CAST (description LIKE '%@%' AS integer) AS email,
         -- Create the phone indicator
         CAST (description LIKE '%___-___-____%' AS integer) AS phone 
    -- What table contains the data? 
    FROM evanston311;

-- Inspect the contents of the new temp table
SELECT *
  FROM indicators limit 3;

 * sqlite:///myfile.db
Done.
Done.
Done.


id,email,phone
1340563,0,0
1826017,0,0
1849204,0,0


In [47]:
%%sql
-- To clear table if it already exists
DROP TABLE IF EXISTS indicators;

-- Create the temp table
CREATE TEMP TABLE indicators AS
  SELECT id, 
         CAST (description LIKE '%@%' AS integer) AS email,
         CAST (description LIKE '%___-___-____%' AS integer) AS phone 
    FROM evanston311;

-- Select the column you'll group by
SELECT priority, 
       -- Compute the proportion of rows with each indicator
       sum(email)/count(*)::numeric AS email_prop, 
       sum(phone)/count(*)::numeric AS phone_prop 
  -- Tables to select from
  FROM evanston311
       LEFT JOIN indicators
       -- Joining condition
       ON evanston311.id=indicators.id
 -- What are you grouping by?
 GROUP BY priority;

 * sqlite:///myfile.db
Done.
Done.
(sqlite3.OperationalError) unrecognized token: ":"
[SQL: -- Select the column you'll group by
SELECT priority, 
       -- Compute the proportion of rows with each indicator
       sum(email)/count(*)::numeric AS email_prop, 
       sum(phone)/count(*)::numeric AS phone_prop 
  -- Tables to select from
  FROM evanston311
       LEFT JOIN indicators
       -- Joining condition
       ON evanston311.id=indicators.id
 -- What are you grouping by?
 GROUP BY priority;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
